# Trabajo Final Inteligencia Artificial I - año 2018: Visión Artificial
---

# Dependencias
---

In [ ]:
import numpy as np

from matplotlib import pyplot as plt
plt.rcParams['image.cmap'] = 'gray'

from skimage import io, color, img_as_float, filters
from skimage.feature import hog

import cv2
import mahotas

## Funciones y clases definidas

- [Representación](./01-representacion.ipynb):

In [ ]:
def img2grey(image, mode='sk'):
    if (mode=='sk'):
        gray = color.rgb2gray(image)
    elif (mode=='cv'):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

def normSize(image, size=(tuple((400, 300)))):
    image = cv2.resize(image, size)
    return image

- [Filtración](./02-filtracion.ipynb):

In [ ]:
def imgClean(image, sigma=1, mode='sk'):
    if (mode == 'sk'):
        clean = filters.gaussian(image, sigma)
    elif (mode == 'cv'):
        clean = cv2.GaussianBlur(image, (3, 3), 0)
    return clean

def imgEdge(image, mode='sk'):
    if (mode == 'sk'):
        edge = filters.sobel(image)
    elif (mode == 'cv'):
        edge = cv2.Laplacian(image, cv2.CV_64F)
    return edge

- [Segmentación](./03-segmentacion.ipynb):

In [ ]:
def threshold(image, mode='sk'):
    if (mode == 'sk'):
        th = filters.threshold_isodata(image)
    elif (mode == 'cv'):
        ret, th = cv2.threshold(image, 0, 255,
                                cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return (image < th)

- [Extracción de características](./04-extraccion.ipynb):

In [ ]:
def m_hog(image):
    feature = hog(image, block_norm='L2-Hys').ravel()
    return feature

def hu_moments(image):
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

def haralick(image):
    feature= mahotas.features.haralick(image).mean(axis=0)
    return feature

def color_histogram(image, mask=None, bins=8):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins],
                        [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    feature = hist.flatten()
    return feature

In [ ]:
def stats(arr):
    
    sum = 0
    for value in arr:
        sum += value
    med = sum / len(arr)
    sum = 0
    for value in arr:
        sum += np.power((value - med), 2)
    dstd = np.sqrt(sum / (len(arr) - 1))
    
    return med, dstd

- [Algoritmos](./05-algoritmos.ipynb):

In [ ]:
class Elemento:
    def __init__(self):
        self.label = None
        self.image = None
        self.feature = []
        self.distance = 0

In [ ]:
def printProgressBar(iteration, total, prefix='', suffix='',
                     decimals=1, length=100, fill='█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of
    decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = (
        "{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end='\r')
    # Print New Line on Complete
    if iteration == total:
        print()

In [ ]:
def ft_extract(image):
    image = normSize(image)
    # aux = img2grey(image, mode='cv')
    # aux = imgClean(aux, mode='cv')
    # aux = imgEdge(aux)
    # aux = threshold(aux, mode='cv')
    image = imgClean(image, mode='cv')
    
    # image_fht = haralick(aux)
    # image_fhm = hu_moments(aux)
    image_fch = color_histogram(image)
    # image_fhog = m_hog(aux)

    # feature = np.hstack([image_fht, image_fhm, image_fhog])
    med, dstd = stats(image_fch)
    # feature = feature.reshape(1, -1)

    return aux, [med, dstd]

# Análisis de la base de datos
---

## Base de datos de entrenamiento

In [ ]:
banana = io.ImageCollection('./data/banana/*.png:./data/banana/*.jpg')
orange = io.ImageCollection('./data/orange/*.png:./data/orange/*.jpg')
lemon = io.ImageCollection('./data/lemon/*.png:./data/lemon/*.jpg')

fig_data, ax = plt.subplots()

data = []
i = 0

# Análisis de bananas en base de datos
iter = 0
printProgressBar(iter, len(banana), prefix='Loading banana data:',
                 suffix='Complete', length=50)
for fruit in banana:
    data.append(Elemento())
    data[i].label = 'banana'
    data[i].image, data[i].feature = ft_extract(fruit)
    # print(data[i].feature.shape)
    ax.plot(data[i].feature[0], data[i].feature[1], 'o', color='yellow')
    i += 1
    iter += 1
    printProgressBar(iter, len(banana), prefix='Loading banana data:',
                     suffix='Complete', length=50)
print("Banana data is ready")

# Análisis de naranjas en base de datos
iter = 0
printProgressBar(iter, len(orange), prefix='Loading orange data:',
                 suffix='Complete', length=50)
for fruit in orange:
    data.append(Elemento())
    data[i].label = 'orange'
    data[i].image, data[i].feature = ft_extract(fruit)
    ax.plot(data[i].feature[0], data[i].feature[1], 'o', color='red')
    i += 1
    iter += 1
    printProgressBar(iter, len(orange), prefix='Loading orange data:',
                     suffix='Complete', length=50)
print("Orange data is ready")

# Análisis de limones en la base de datos
iter = 0
printProgressBar(iter, len(lemon), prefix='Loading lemon data:',
                 suffix='Complete', length=50)
for fruit in lemon:
    data.append(Elemento())
    data[i].label = 'lemon'
    data[i].image, data[i].feature = ft_extract(fruit)
    ax.plot(data[i].feature[0], data[i].feature[1], 'o', color='blue')
    i += 1
    iter += 1
    printProgressBar(iter, len(lemon), prefix='Loading lemon data:',
                     suffix='Complete', length=50)
print("Lemon data is ready")

print("Data analysis completed")

## Base de datos de testeo

In [ ]:
banana_test = io.ImageCollection('./test/banana/*.png:./test/banana/*.jpg')
orange_test = io.ImageCollection('./test/orange/*.png:./test/orange/*.jpg')
lemon_test = io.ImageCollection('./test/lemon/*.png:./test/lemon/*.jpg')

fig_test, ax = plt.subplots()

test = []
i = 0

# Análisis de bananas en base de datos
iter = 0
printProgressBar(iter, len(banana_test), prefix='Loading banana test data:',
                 suffix='Complete', length=50)
for fruit in banana_test:
    test.append(Elemento())
    test[i].label = 'banana'
    test[i].image, test[i].feature = ft_extract(fruit)
    # print(data[i].feature.shape)
    ax.plot(test[i].feature[0], test[i].feature[1], 'o', color='yellow')
    i += 1
    iter += 1
    printProgressBar(iter, len(banana_test), prefix='Loading banana test data:',
                     suffix='Complete', length=50)
print("Banana test data is ready")

# Análisis de naranjas en base de datos
iter = 0
printProgressBar(iter, len(orange_test), prefix='Loading orange test data:',
                 suffix='Complete', length=50)
for fruit in orange_test:
    test.append(Elemento())
    test[i].label = 'orange'
    test[i].image, test[i].feature = ft_extract(fruit)
    ax.plot(test[i].feature[0], test[i].feature[1], 'o', color='red')
    i += 1
    iter += 1
    printProgressBar(iter, len(orange_test), prefix='Loading orange test data:',
                     suffix='Complete', length=50)
print("Orange test data is ready")

# Análisis de limones en la base de datos
iter = 0
printProgressBar(iter, len(lemon_test), prefix='Loading lemon test data:',
                 suffix='Complete', length=50)
for fruit in lemon_test:
    test.append(Elemento())
    test[i].label = 'lemon'
    test[i].image, test[i].feature = ft_extract(fruit)
    ax.plot(test[i].feature[0], test[i].feature[1], 'o', color='blue')
    i += 1
    iter += 1
    printProgressBar(iter, len(lemon_test), prefix='Loading lemon test data:',
                     suffix='Complete', length=50)
print("Lemon test data is ready")

print("Test data analysis completed")

# Resultados con algoritmo K-nn
---

Con $K=1$:

In [ ]:
correct = 0

printProgressBar(correct, len(test), prefix='Correct predictions:',
                 suffix='Total', length=50)

for t in test:

    i = 0
    sum = 0
    for ft in data[0].feature:
            sum = sum + np.power(np.abs(t.feature[i] - ft), 2)
            i += 1
    d = np.sqrt(sum)

    for element in data:
        sum = 0
        i = 0
        for ft in (element.feature):
            sum = sum + np.power(np.abs((t.feature[i]) - ft), 2)
            i += 1

        element.distance = np.sqrt(sum)

        if (sum < d):
            d = sum
            label = element.label
    
    if (t.label == label):
        correct += 1
    printProgressBar(correct, len(test), prefix='Correct predictions:',
                 suffix='Total', length=50)

Con $K=10$, y dándole un $30\%$ de peso al más cercano:

In [ ]:
# NOTA IMPORTANTE: Antes de ejecutar este bloque 
# actualizar base de datos test

k = 10

correct = 0

printProgressBar(correct, len(test), prefix='Correct predictions:',
                 suffix='Total', length=50)

for t in test:

    for element in data:
        sum = 0
        i = 0
        for ft in (element.feature):
            sum = sum + np.power(np.abs((t.feature[i]) - ft), 2)
            i += 1

        element.distance = np.sqrt(sum)

    # Bubblesort
    swap = True
    while (swap):
        swap = False
        for i in range(1, len(data)-1) :
            if (data[i-1].distance > data[i].distance):
                aux = data[i]
                data[i] = data[i-1]
                data[i-1] = aux
                swap = True
    
    eval = [0, 0, 0]
    
    if (data[0].label == 'banana'):
        eval[0] += 10
        
    if (data[0].label == 'orange'):
        eval[1] += 10
        
    if (data[0].label == 'lemon'):
        eval[2] += 10
        
    for i in range(1, k):
        
        if (data[i].label == 'banana'):
            eval[0] += 10

        if (data[i].label == 'orange'):
            eval[1] += 10

        if (data[i].label == 'lemon'):
            eval[2] += 10
    
    aux = eval[0]
    if (aux < eval[1]):
        aux = eval[1]
    if (aux < eval[2]):
        aux = eval[2]
    
    if (aux == eval[0]):
        label = 'banana'
    if (aux == eval[1]):
        label = 'orange'
    if (aux == eval[2]):
        label = 'lemon'
    
    if (t.label == label):
        correct += 1
    printProgressBar(correct, len(test), prefix='Correct predictions:',
                 suffix='Total', length=50)

## Rendimiento en función de K
Para ejecutar el siguiente código, cambiar kernel a un kernel de bash.

In [15]:
cd
cd Desktop/TPFinal-IA1/fruit-classifier/src

In [16]:
if [ -f ./data.pkl ]; then
    echo "Old data found. Removing data.pkl."
    rm data.pkl
fi
echo "Data analysis..."
python data.py data

if [ -f ./test.pkl ]; then
    echo "Old test data found. Removing test.pkl."
    rm test.pkl
fi
echo "Test data analysis..."
python test.py

Old data found. Removing data.pkl.
Data analysis...
Loading banana data: |██████████████████████████████████████████████████| 100.0% Complete
Banana data is ready
Loading orange data: |██████████████████████████████████████████████████| 100.0% Complete
Orange data is ready
Loading lemon data: |██████████████████████████████████████████████████| 100.0% Complete
Lemon data is ready
Data analysis completed
Old test data found. Removing test.pkl.
Test data analysis...
Loading banana test data: |██████████████████████████████████████████████████| 100.0% Complete
Banana test data is ready
Loading orange test data: |██████████████████████████████████████████████████| 100.0% Complete
Orange test data is ready
Loading lemon test data: |██████████████████████████████████████████████████| 100.0% Complete
Lemon test data is ready
Test data analysis completed


In [14]:
x=1
while [ $x -le 20 ]
do
    echo "Running with K = $x"
    python knn-testing.py $x
    x=$(( $x + 1 ))
done

Running with K = 1
307
295 297 307
Running with K = 2
307
295 297 307
Running with K = 3
307
295 297 307
Running with K = 4
306
294 294 306
Running with K = 5
301
291 296 301
Running with K = 6
309
299 299 309
Running with K = 7
308
300 297 308
Running with K = 8
307
296 292 307
Running with K = 9
304
131 95 78
Running with K = 10
307
131 99 84
Running with K = 11
304
131 93 80
Running with K = 12
304
131 100 84
Running with K = 13
302
131 94 77
Running with K = 14
300
131 100 81
Running with K = 15
303
131 96 77
Running with K = 16
299
131 101 79
Running with K = 17
302
131 99 74
Running with K = 18
301
131 103 78
Running with K = 19
304
131 96 78
Running with K = 20
304
131 100 79


# Resultados con algoritmo K-means
---

## Entrenamiento del algoritmo

In [ ]:
import random

banana_data = []
orange_data = []
lemon_data = []

for element in data:
    if (element.label == 'banana'):
        banana_data.append(element)
    if (element.label == 'orange'):
        orange_data.append(element)
    if (element.label == 'lemon'):
        lemon_data.append(element)

b_mean = list(random.choice(banana_data).feature)
o_mean = list(random.choice(orange_data).feature)
l_mean = list(random.choice(lemon_data).feature)

b_flag = True
o_flag = True
l_flag = True

b_len = [0, 0]
o_len = [0, 0]
l_len = [0, 0]

iter = 0

# while (b_flag or o_flag or l_flag):
while (iter < 20):
    
    banana_data = []
    orange_data = []
    lemon_data = []

    # ASIGNACION

    for element in data:
        sum_b = 0
        sum_o = 0
        sum_l = 0

        for i in range(0, len(element.feature)-1):
            sum_b += np.power(np.abs(b_mean[i] - element.feature[i]), 2)
            sum_o += np.power(np.abs(o_mean[i] - element.feature[i]), 2)
            sum_l += np.power(np.abs(l_mean[i] - element.feature[i]), 2)

        dist_b = np.sqrt(sum_b)
        dist_o = np.sqrt(sum_o)
        dist_l = np.sqrt(sum_l)
        
        aux = dist_b
        if (dist_o < aux):
            aux = dist_o
        if (dist_l < aux):
            aux = dist_l
            
        if (aux == dist_b):
            banana_data.append(element.feature)
        elif (aux == dist_o):
            orange_data.append(element.feature)
        elif(aux == dist_l):
            lemon_data.append(element.feature)
            
    # ACTUALIZACION
    sum_b = [0, 0]
    for b in banana_data:
        sum_b[0] += b[0]
        sum_b[1] += b[1]
        
    sum_o = [0, 0]
    for o in orange_data:
        sum_o[0] += o[0]
        sum_o[1] += o[1]
        
    sum_l = [0, 0]
    for l in lemon_data:
        sum_l[0] += l[0]
        sum_l[1] += l[1]
    
    b_mean[0] = sum_b[0] / len(banana_data)
    b_mean[1] = sum_b[1] / len(banana_data)
    o_mean[0] = sum_o[0] / len(orange_data)
    o_mean[1] = sum_o[1] / len(orange_data)
    l_mean[0] = sum_l[0] / len(lemon_data)
    l_mean[1] = sum_l[1] / len(lemon_data)
    
    # print(len(banana_data), len(orange_data), len(lemon_data))
    
    # CONVERGENCIA Y CONDICIÓN DE SALIDA
    print(len(banana_data), len(orange_data), len(lemon_data))
    
    if (b_mean == b_len):
        b_flag = False
    else:
        b_len = b_mean
        
    if (o_mean == o_len):
        o_flag = False
    else:
        o_len = o_mean

    if (l_mean == l_len):
        l_flag = False
    else:
        l_len = l_mean
    
    iter += 1

## Evaluación de rendimiento

In [ ]:
correct = 0

printProgressBar(correct, len(test), prefix='Correct predictions:',
                 suffix='Total', length=50)

for t in test:

    sum_b = 0
    sum_o = 0
    sum_l = 0

    for i in range(0, len(t.feature)-1):
        sum_b += np.power(np.abs(t.feature[i] - b_mean[i]), 2)
        sum_o += np.power(np.abs(t.feature[i] - o_mean[i]), 2)
        sum_l += np.power(np.abs(t.feature[i] - l_mean[i]), 2)

    dist_b = np.sqrt(sum_b)
    dist_o = np.sqrt(sum_o)
    dist_l = np.sqrt(sum_l)
    # print(dist_b, dist_o, dist_l)
    
    aux = dist_b
    if (dist_o < aux):
        aux = dist_o
    if (dist_l < aux):
        aux = dist_l

    if (aux == dist_b):
        label = 'banana'
    if (aux == dist_o):
        label = 'orange'
    if (aux == dist_l):
        label = 'lemon'

    if (t.label == label):
        correct += 1
    
    printProgressBar(correct, len(test), prefix='Corrects prediction:',
                 suffix='Total', length=50)